In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
with tf.device('/GPU:0'):
               a = tf.constant([[1.0,2.0,3.0]])
               b = tf.constant([[1.0], [2.0], [3.0]])
               result = tf.matmul(a, b)
print(result)

tf.Tensor([[14.]], shape=(1, 1), dtype=float32)


In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
print(tf.test.gpu_device_name())
print("Num GPUs Available: ",len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)


Num GPUs Available:  0
2.17.0


In [4]:
data_dir = 'C:/Users/mauli/Desktop/SOIL/MAIN/train'

In [5]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2,
    horizontal_flip = False,
    zoom_range = 0.2,
    shear_range = 0.2,
)

train_generator =datagen.flow_from_directory(
    data_dir,
    target_size = (224, 224),
    batch_size =32,
    class_mode = 'categorical',
    subset = 'training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size = (224, 224),
    batch_size =32,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 6146 images belonging to 8 classes.
Found 1532 images belonging to 8 classes.


In [6]:
 # Load the DenseNet121 model without the top fully connected layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Add custom layers on top of the base model
model = models.Sequential([
    base_model,  

    layers.Flatten(),
    
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(8, activation='softmax')
])

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        epochs=5,  # You can adjust the number of epochs
        validation_data=validation_generator
    )

Epoch 1/5


C:\Users\mauli\Desktop\python_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


193/193 ━━━━━━━━━━━━━━━━━━━━ 599s 3s/step - accuracy: 0.6772 - loss: 4.9288 - val_accuracy: 0.8336 - val_loss: 0.5819
Epoch 2/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 512s 3s/step - accuracy: 0.8727 - loss: 0.3670 - val_accuracy: 0.8048 - val_loss: 0.7953
Epoch 3/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - accuracy: 0.9031 - loss: 0.3171 - val_accuracy: 0.8512 - val_loss: 0.6037
Epoch 4/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 523s 3s/step - accuracy: 0.9121 - loss: 0.2701 - val_accuracy: 0.8610 - val_loss: 0.8453
Epoch 5/5
193/193 ━━━━━━━━━━━━━━━━━━━━ 523s 3s/step - accuracy: 0.9327 - loss: 0.2183 - val_accuracy: 0.8805 - val_loss: 0.5524


In [10]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_accuracy:.2f}')

48/48 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.8890 - loss: 0.5371
Validation Accuracy: 0.88


In [11]:
import pandas as pd

In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize image to match model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to match training data normalization
    return img_array

def classify_image(model, img_path, class_indices):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    class_label = {v: k for k, v in class_indices.items()}[predicted_class]  # Convert index to class name
    return class_label, predictions[0]

img_path = 's.jpg'
class_indices = train_generator.class_indices  # Mapping of classes to indices
class_label, prediction = classify_image(model, img_path, class_indices)

print(f"Predicted Class: {class_label}")
print("Prediction Probabilities:", prediction)

model.save('cnn_new_saved_model.h5', save_format='h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step


Predicted Class: Alluvial soil
Prediction Probabilities: [9.56986964e-01 3.51464085e-04 4.10718583e-02 1.13425685e-07
 8.84739042e-04 6.54944859e-04 4.10779285e-05 8.91476066e-06]
